In [ ]:
#first we need to make our dataset
"""
A photo of a 25 year old woman from Germany

running this prompt for every single woman i have 


"""

In [2]:
#create the dataset of prompts 
import pandas as pd

# Download the mapping.csv file from GitHub
mapping_url = "https://raw.githubusercontent.com/flekschas/simple-world-map/master/mapping.csv"
mapping_df = pd.read_csv(mapping_url)


In [3]:
# Get list of country names
countries_list = mapping_df['Country Name'].tolist()

# Or if you want country codes instead
country_codes = mapping_df['Country Code'].tolist()

# If you want both in a list of tuples
countries_with_codes = list(zip(mapping_df['Country Name'], mapping_df['Country Code']))

In [4]:
print("Total number of countries:", len(countries_list))
print("\nFirst 5 countries:")
print(countries_list)

Total number of countries: 175

First 5 countries:
['Somaliland', 'United Arab Emirates', 'Afghanistan', 'Albania', 'Armenia', 'Angola', 'Argentina', 'Austria', 'Australia', 'Azerbaijan', 'Bosnia and Herzegowina', 'Bangladesh', 'Belgium', 'Burkina Faso', 'Bulgaria', 'Burundi', 'Benin', 'Brunei', 'Bolivia', 'Brazil', 'Bahamas', 'Bhutan', 'Botswana', 'Belarus', 'Belize', 'Canada', 'Democratic Republic of the Congo', 'Central African Republic', 'Congo', 'Switzerland', "Cote d'Ivoire", 'Chile', 'Cameroon', 'China', 'Colombia', 'Costa Rica', 'Cuba', 'Cyprus', 'Czech', 'Germany', 'Djibouti', 'Denmark', 'Dominican Republic', 'Algeria', 'Ecuador', 'Estonia', 'Egypt', 'West Sahara', 'Eritrea', 'Spain', 'Ethiopia', 'Finland', 'Fiji', 'Falkland Islands', 'France', 'Gabon', 'United Kingdom', 'Georgia', 'Ghana', 'Greenland', 'Gambia', 'Guinea', 'Equatorial Guinea', 'Greece', 'Guatemala', 'Guinea-Bissau', 'Guyana', 'Honduras', 'Croatia', 'Haiti', 'Hungary', 'Indonesia', 'Ireland', 'Israel', 'India',

In [5]:
# Create prompts list using list comprehension
prompts_list = [f"A photo of a 25 year old woman from {country}" for country in countries_list]

# Print a few examples to verify
print("Sample prompts:")
print("\n".join(prompts_list[:5]))

Sample prompts:
A photo of a 25 year old woman from Somaliland
A photo of a 25 year old woman from United Arab Emirates
A photo of a 25 year old woman from Afghanistan
A photo of a 25 year old woman from Albania
A photo of a 25 year old woman from Armenia


In [13]:
# Test cell for single image generation
from dotenv import load_dotenv
import os
import replicate

# Load environment variables
load_dotenv()
token = os.getenv('REPLICATE_API_TOKEN')

# Test with a single country
test_country = "Germany"
test_prompt = f"A photo of a 25 year old woman from {test_country}"

# Create test directory
os.makedirs('generated_images', exist_ok=True)

# Run single test
try:
    print(f"Testing with prompt: {test_prompt}")
    
    response = replicate.run(
        "black-forest-labs/flux-1.1-pro-ultra",
        input={
            "prompt": test_prompt,
            "aspect_ratio": "3:2",
            "raw": True,
            "output_format": "jpg",
            "safety_tolerance": 2,
            "image_prompt_strength": 0.1
        }
    )
    
    # Save test image
    filename = f"generated_images/test_{test_country.replace(' ', '_')}3.jpg"
    with open(filename, "wb") as file:
        file.write(response.read())
    
    print(f"Success! Test image saved as: {filename}")
except Exception as e:
    print(f"Error during test: {str(e)}")

Testing with prompt: A photo of a 25 year old woman from Germany
Success! Test image saved as: generated_images/test_Germany3.jpg


In [6]:
import asyncio
import nest_asyncio
import aiohttp
import os
from tqdm import tqdm
import time

from dotenv import load_dotenv
import os
import replicate

# Load environment variables from .env file
load_dotenv()

# Now you can access the token from environment variables
# The token will be automatically loaded from .env file
token = os.getenv('REPLICATE_API_TOKEN')

async def generate_image(session, country, prompt):
    try:
        input_data = {
            "prompt": prompt,
            "aspect_ratio": "3:2",
            "raw": True,
            "output_format": "jpg",
            "safety_tolerance": 2,
            "image_prompt_strength": 0.1
        }
        
        # Use regular run instead of async_run
        response = replicate.run(
            "black-forest-labs/flux-1.1-pro-ultra",
            input=input_data
        )
        
        # Create images directory if it doesn't exist
        os.makedirs('generated_images', exist_ok=True)
        
        # Save image with country name
        filename = f"generated_images/{country.replace(' ', '_')}.jpg"
        with open(filename, "wb") as file:
            file.write(response.read())
            
        return True, country
        
    except Exception as e:
        print(f"Error for {country}: {str(e)}")  # Add error printing
        return False, country

async def process_batch(prompts_batch):
    async with aiohttp.ClientSession() as session:
        tasks = []
        for country, prompt in prompts_batch:
            task = generate_image(session, country, prompt)
            tasks.append(task)
        
        results = await asyncio.gather(*tasks, return_exceptions=True)
        
        # Track failed generations
        failed_countries = [country for success, country in results if not success]
        if failed_countries:
            print(f"Failed generations for countries: {', '.join(failed_countries)}")
        
        return failed_countries

async def main():
    # Create pairs of (country, prompt)
    country_prompt_pairs = list(zip(countries_list, prompts_list))
    
    # Process in batches of 10
    batch_size = 10
    failed_countries = []
    
    # Create progress bar
    with tqdm(total=len(country_prompt_pairs), desc="Generating images") as pbar:
        for i in range(0, len(country_prompt_pairs), batch_size):
            batch = country_prompt_pairs[i:i + batch_size]
            batch_failed = await process_batch(batch)
            failed_countries.extend(batch_failed)
            pbar.update(len(batch))
            
            # Add a small delay between batches to avoid rate limiting
            await asyncio.sleep(1)
    
    # Final report
    print("\nImage generation complete!")
    if failed_countries:
        print(f"\nFailed generations for {len(failed_countries)} countries:")
        print("\n".join(failed_countries))
    else:
        print("\nAll images generated successfully!")

import nest_asyncio
nest_asyncio.apply()  # This allows async code to run in Jupyter

# Now run the async code
start_time = time.time()
await main()  # Use await instead of asyncio.run()
end_time = time.time()
print(f"\nTotal execution time: {end_time - start_time:.2f} seconds")

Generating images:   0%|          | 0/175 [00:00<?, ?it/s]

Generating images: 100%|██████████| 175/175 [27:42<00:00,  9.50s/it]


Image generation complete!

All images generated successfully!

Total execution time: 1662.75 seconds
